In [1]:
pathtorepo = "C:\\Users\\Alex\\Desktop\\repo\\2612\\0903\\"

using Pkg
Pkg.activate(pathtorepo * "dynamical-systems\\env\\integrate\\")
include("C:\\Users\\Alex\\Desktop\\repo\\2612\\0903\\dynamical-systems\\system.jl")
include("C:\\Users\\Alex\\Desktop\\repo\\2612\\0903\\dynamical-systems\\visual_regimes.jl");

  Activating project at `C:\Users\Alex\Desktop\repo\2612\0903\dynamical-systems\env\integrate`


In [2]:
using StaticArrays, DifferentialEquations, DynamicalSystems
E, x, y  = 0..40, 0..1,  0..1
box = E × x × y
using CairoMakie, GLMakie;

In [30]:
const τ = 0.013;  const τD = 0.07993;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4
const α = 1.58; const ΔU0 = 0.305; 

In [31]:
time = 1000; tt = 1000; tstep = 0.001; times = [time, tt]
integ_set = (alg = Vern9(), adaptive = false, dt = tstep)

time_int_3d = [100000, 200000]
eps_vec = [1e-10, 0.0, 0.0];

In [59]:
U0 = 0.265;
I0 = -1.706
p = [α, τ, τD, τy, J, xthr, ythr, U0, ΔU0, β, I0]
u0_1 = output[3][end] #[14.638526524981895, 0.4320692774159767, 0.40718205935401675]
u0_2 = output[4][end] #[14.971143689707221, 0.4053540799922068, 0.42133199265006]
println("last point  pink trajectory:$u0_1")
println("black trajectory:$u0_2")

last point  pink trajectory:[4.640991536943699, 0.789276664749172, 0.4416534220430231]
black trajectory:[3.9592581233157085, 0.8172526533572356, 0.44681873976636194]


In [60]:
output = research_regime_2_traj(TM, u0_1, u0_2, p, times, integ_set,
        t_int_3d = time_int_3d,
        plot = false, onlylastpoint = false, dis_hc = true)

9-element Vector{Any}:
  3-dimensional CoupledODEs
  3-dimensional CoupledODEs
  3-dimensional StateSpaceSet{Float64} with 1000001 points
  3-dimensional StateSpaceSet{Float64} with 1000001 points
 0.00030125334462359845
 0.0003012471196834236
  Figure()
  3-dimensional StateSpaceSet{Float64} with 1 points
  Vector{ComplexF64}[[-6.39202144621012 + 0.0im, 2.1633886996418656 - 13.783195438835794im, 2.1633886996418656 + 13.783195438835794im]]

In [61]:
prev_1 = u0_1
prev_2 = u0_2
println("I0: $I0")
println("previous last point  pink trajectory:$prev_1")
println("previous black trajectory:$prev_2")

I0: -1.706
previous last point  pink trajectory:[4.640991536943699, 0.789276664749172, 0.4416534220430231]
previous black trajectory:[3.9592581233157085, 0.8172526533572356, 0.44681873976636194]


In [62]:
data_1 = output[3]
data_2 = output[4];

In [63]:
# 3d interactive
tstart, tfinish = 1, 1000000

f = Figure(resolution = (700, 700))
ax = LScene(f[1, 1], show_axis = true)


scale!(ax.scene, 50, 50, 1)

lines!(data_2[time_int_3d[1]:time_int_3d[2], 3], data_2[time_int_3d[1]:time_int_3d[2], 2],
        data_2[time_int_3d[1]:time_int_3d[2], 1], linewidth = 1.0, color = :black)
        
lines!(data_1[tstart:tfinish, 3], data_1[tstart:tfinish, 2],
        data_1[tstart:tfinish, 1], linewidth = 1.0, color = :deeppink)
        
scatter!(output[8][1][3], output[8][1][2], output[8][1][1], markersize = 10.0, color = :orange)

display(GLMakie.Screen(), f)

GLMakie.Screen(...)

In [64]:
ts, tf = 20000, 30000
f = Figure(resolution = (1200, 350))
axis = Axis(f[1, 1])
lines!(output[4][ts:tf, 1])
display(f)

GLMakie.Screen(...)

In [65]:
pink = lyapunovspectrum(output[1], time; Ttr = tt)
black = lyapunovspectrum(output[2], time; Ttr = tt)
println("LS  pink trajectory:$pink")
println("LS  black trajectory:$black")

LS  pink trajectory:[0.00015159295493105018, -0.029520575399908475, -4.681802154506056]
LS  black trajectory:[0.0023281266227962523, -0.020762100374911004, -4.684042796636224]
